## Black Box Approach for all 10 genres classification

We aim at training a Neural Network to distinguish between all the different genres

In [1]:
#importing google drive
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
#setting the working directory
%cd /gdrive/MyDrive/polimi/NAML/NAML_proj/

/gdrive/MyDrive/polimi/NAML/NAML_proj


In [3]:
import tensorflow as tf
from PIL import Image
import numpy as np
import os
import random
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
import librosa
import os

tfk = tf.keras
tfkl = tf.keras.layers
seed = 42

The dataset is composed of Mel-Spectrograms of each audio sample, labeled with respect to each genre

In [4]:
dataset = []
genres = {'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 
        'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}
n_genres = 10

for genre, genre_number in genres.items():
    for filename in os.listdir(f'dataset_old/genres/{genre}'):
        songname = f'dataset_old/genres/{genre}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=29.7)
        ps = librosa.feature.melspectrogram(y=y, sr=sr, hop_length = 256, n_fft = 512)
        ps = librosa.power_to_db(ps**2)
        dataset.append( (ps, genre_number) )
    print(str(genre+' done'))

blues done
classical done
country done
disco done
hiphop done
jazz done
metal done
pop done
reggae done
rock done


We split the dataset according to the following composition:


*   70% training set
*   20% validation set
*   10% test set

Maintaining equal proportions amongst classes



In [5]:
order = np.arange(start = 0, stop = 100, step = 1)

np.random.seed(seed)

training = []
validation = []
test = []

for i in range(n_genres):
  shuffle = np.random.permutation(order)
  for k in range(70):
    training.append(dataset[i*100 + shuffle[k]])
  for l in range(20):
    validation.append(dataset[i*100 + shuffle[l+70]])
  for m in range(10):
    test.append(dataset[i*100 + shuffle[m+90]])

In [6]:
X_train, Y_train = zip(*training)
X_valid, Y_valid = zip(*validation)
X_test, Y_test = zip(*test)

X_train = np.array([x.reshape( (128, 2559, 1) ) for x in X_train])
X_valid = np.array([x.reshape( (128, 2559, 1) ) for x in X_valid])
X_test = np.array([x.reshape( (128, 2559, 1) ) for x in X_test])


Y_train = np.array(tfk.utils.to_categorical(Y_train, n_genres))
Y_valid = np.array(tfk.utils.to_categorical(Y_valid, n_genres))
Y_test = np.array(tfk.utils.to_categorical(Y_test, n_genres))

The proposed model is composed as a stack of convolutional layer, followed by a Global Average Pooling layer leading to a fully connected section.

Optimization is performed as a Batch version of Adam optimizer algorithm

In [7]:
def build_model(input_shape, n_units):
# Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    conv1 = tfkl.Conv2D(
        name = 'conv_1',
        filters=4,
        kernel_size=(5, 5),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(input_layer)
    conv1_2 = tfkl.Conv2D(
        name = 'conv_1_2',
        filters=8,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(conv1)
    pool1 = tfkl.MaxPooling2D(
        name = 'pool_1',
        pool_size = (2, 2)
    )(conv1_2)

    conv2 = tfkl.Conv2D(
        name = 'conv_2',
        filters=16,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool1)
    conv2_2 = tfkl.Conv2D(
        name = 'conv_2_2',
        filters=16,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(conv2)
    pool2 = tfkl.MaxPooling2D(
        name = 'pool_2',
        pool_size = (2, 2)
    )(conv2_2)

    conv3 = tfkl.Conv2D(
        name = 'conv_3',
        filters=32,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool2)
    pool3 = tfkl.MaxPooling2D(
        name = 'pool_3',
        pool_size = (2, 2)
    )(conv3)

    conv4 = tfkl.Conv2D(
        name = 'conv_4',
        filters=64,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool3)
    pool4 = tfkl.MaxPooling2D(
        name = 'pool_4',
        pool_size = (2, 2)
    )(conv4)

    conv5 = tfkl.Conv2D(
        name = 'conv_5',
        filters=128,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool4)
    pool5 = tfkl.MaxPooling2D(
        name = 'pool_5',
        pool_size = (2, 2)
    )(conv5)

    conv6 = tfkl.Conv2D(
        name = 'conv_6',
        filters=286,
        kernel_size=(1, 1),
        strides = (1, 1),
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed)
    )(pool5)
    
    global_average = tfkl.GlobalAveragePooling2D(name = 'GAP')(conv6)
    global_average = tfkl.Dropout(0.3, seed=seed, name = 'Dropout_GAP')(global_average)
    
    classifier_layer = tfkl.Dense(units=64, name='Classifier', activation='relu')(global_average)
    
    classifier_layer = tfkl.Dropout(0.3, seed=seed, name = 'Dropout_Classifier')(classifier_layer)
    classifier_layer_2 = tfkl.Dense(units=32, name='Classifier_2', activation='relu')(classifier_layer)
    classifier_layer_2 = tfkl.Dropout(0.3, seed=seed, name = 'Dropout_Classifier_2')(classifier_layer_2)
    output_layer = tfkl.Dense(units=n_units, activation='softmax', name='Output')(classifier_layer_2)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model

    return model

In [8]:
metrics_accuracy = []
metrics_precision = []
metrics_recall = []
metrics_f1 = []

n_trials = 10
input_shape = (128, 2559, 1)
early_stopping = tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True)
adaptive_LR = tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-4)

for i in range(n_trials):
  model = build_model(input_shape, n_genres)
  standard_history = model.fit(
    x = X_train,
    y = Y_train,
    epochs = 500,
    batch_size = 100,
    validation_data= (X_valid, Y_valid),
    callbacks = [early_stopping, adaptive_LR]
    )
  predictions = model.predict(X_test)
  metrics_accuracy.append(accuracy_score(np.argmax(Y_test, axis=-1), np.argmax(predictions, axis=-1)))
  metrics_precision.append(precision_score(np.argmax(Y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro'))
  metrics_recall.append(recall_score(np.argmax(Y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro'))
  metrics_f1.append(f1_score(np.argmax(Y_test, axis=-1), np.argmax(predictions, axis=-1), average='macro'))
  model.save(str('/gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_0_'+str(i)))
  

Epoch 1/500
7/7 [==============================] - 29s 2s/step - loss: 2.3472 - accuracy: 0.1043 - val_loss: 2.3013 - val_accuracy: 0.1000 - lr: 0.0010
Epoch 2/500
7/7 [==============================] - 10s 1s/step - loss: 2.3076 - accuracy: 0.1000 - val_loss: 2.3005 - val_accuracy: 0.0950 - lr: 0.0010
Epoch 3/500
7/7 [==============================] - 10s 1s/step - loss: 2.3008 - accuracy: 0.1086 - val_loss: 2.2990 - val_accuracy: 0.0500 - lr: 0.0010
Epoch 4/500
7/7 [==============================] - 10s 1s/step - loss: 2.2981 - accuracy: 0.1029 - val_loss: 2.2923 - val_accuracy: 0.0800 - lr: 0.0010
Epoch 5/500
7/7 [==============================] - 10s 1s/step - loss: 2.2864 - accuracy: 0.1343 - val_loss: 2.2756 - val_accuracy: 0.0700 - lr: 0.0010
Epoch 6/500
7/7 [==============================] - 10s 1s/step - loss: 2.2733 - accuracy: 0.1000 - val_loss: 2.2393 - val_accuracy: 0.0800 - lr: 0.0010
Epoch 7/500
7/7 [==============================] - 10s 1s/step - loss: 2.2448 - accuracy

INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_0_0/assets


INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_0_0/assets


Epoch 1/500
7/7 [==============================] - 12s 2s/step - loss: 2.3556 - accuracy: 0.1014 - val_loss: 2.2980 - val_accuracy: 0.1050 - lr: 0.0010
Epoch 2/500
7/7 [==============================] - 10s 1s/step - loss: 2.3096 - accuracy: 0.0929 - val_loss: 2.3004 - val_accuracy: 0.1550 - lr: 0.0010
Epoch 3/500
7/7 [==============================] - 10s 1s/step - loss: 2.2983 - accuracy: 0.1157 - val_loss: 2.2990 - val_accuracy: 0.1000 - lr: 0.0010
Epoch 4/500
7/7 [==============================] - 10s 1s/step - loss: 2.3015 - accuracy: 0.1114 - val_loss: 2.2929 - val_accuracy: 0.1550 - lr: 0.0010
Epoch 5/500
7/7 [==============================] - 10s 1s/step - loss: 2.2861 - accuracy: 0.1457 - val_loss: 2.2486 - val_accuracy: 0.1550 - lr: 0.0010
Epoch 6/500
7/7 [==============================] - 10s 1s/step - loss: 2.2480 - accuracy: 0.1643 - val_loss: 2.1723 - val_accuracy: 0.1500 - lr: 0.0010
Epoch 7/500
7/7 [==============================] - 10s 1s/step - loss: 2.2312 - accuracy

INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_0_1/assets


...

INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_0_8/assets


INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_0_8/assets


Epoch 1/500
7/7 [==============================] - 11s 2s/step - loss: 2.3257 - accuracy: 0.0929 - val_loss: 2.2982 - val_accuracy: 0.2050 - lr: 0.0010
Epoch 2/500
7/7 [==============================] - 10s 1s/step - loss: 2.3016 - accuracy: 0.1186 - val_loss: 2.2946 - val_accuracy: 0.1850 - lr: 0.0010
Epoch 3/500
7/7 [==============================] - 10s 1s/step - loss: 2.2949 - accuracy: 0.1029 - val_loss: 2.2882 - val_accuracy: 0.1150 - lr: 0.0010
Epoch 4/500
7/7 [==============================] - 10s 1s/step - loss: 2.2929 - accuracy: 0.1243 - val_loss: 2.2570 - val_accuracy: 0.1750 - lr: 0.0010
Epoch 5/500
7/7 [==============================] - 10s 1s/step - loss: 2.2668 - accuracy: 0.1514 - val_loss: 2.1985 - val_accuracy: 0.2050 - lr: 0.0010
Epoch 6/500
7/7 [==============================] - 10s 1s/step - loss: 2.2445 - accuracy: 0.1657 - val_loss: 2.1284 - val_accuracy: 0.2050 - lr: 0.0010
Epoch 7/500
7/7 [==============================] - 10s 1s/step - loss: 2.1760 - accuracy

INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_0_9/assets


INFO:tensorflow:Assets written to: /gdrive/MyDrive/polimi/NAML/NAML_proj/models/stability/complete_0_9/assets


In [14]:
metrics_accuracy = np.array(metrics_accuracy)
metrics_precision = np.array(metrics_precision)
metrics_recall = np.array(metrics_recall)
metrics_f1 = np.array(metrics_f1)

print(str('mean accuracy = ' + str(metrics_accuracy.mean() )))
print(str('mean precision = ' + str(metrics_precision.mean() )))
print(str('mean recall = ' + str(metrics_recall.mean() )))
print(str('mean f1 = ' + str(metrics_f1.mean() )))

print('***___***')

print(str('accuracy standard deviation = ' + str(metrics_accuracy.std() )))
print(str('precision standard deviation = ' + str(metrics_precision.std() )))
print(str('recall standard deviation = ' + str(metrics_recall.std() )))
print(str('f1 standard deviation = ' + str(metrics_f1.std() )))

print('***___***')

print(str('accuracy range = ' + str(metrics_accuracy.max() - metrics_accuracy.min() )))
print(str('precision range = ' + str(metrics_precision.max() - metrics_precision.min() )))
print(str('recall range = ' + str(metrics_recall.max() - metrics_recall.min() )))
print(str('f1 range = ' + str(metrics_f1.max() - metrics_f1.min() )))

print('***___***')

print(str('min accuracy = ' + str(metrics_accuracy.min() )))
print(str('min precision = ' + str(metrics_precision.min() )))
print(str('min recall = ' + str(metrics_recall.min() )))
print(str('min f1 = ' + str(metrics_f1.min() )))



mean accuracy = 0.734
mean precision = 0.744294511044511
mean recall = 0.734
mean f1 = 0.7267590308496487
***___***
accuracy standard deviation = 0.03611094017053559
precision standard deviation = 0.04364789087156824
recall standard deviation = 0.03611094017053554
f1 standard deviation = 0.03861747903338075
***___***
accuracy range = 0.13
precision range = 0.15380591630591622
recall range = 0.1299999999999999
f1 range = 0.14497740426068595
***___***
min accuracy = 0.68
min precision = 0.6765476190476192
min recall = 0.68
min f1 = 0.664448009506833


In [10]:
print(metrics_accuracy)
print(metrics_precision)
print(metrics_recall)
print(metrics_f1)

[0.72 0.81 0.68 0.71 0.72 0.71 0.76 0.71 0.75 0.77]
[0.72744589 0.83035354 0.67654762 0.71777778 0.73650794 0.71015956
 0.79651154 0.71793706 0.74787879 0.7818254 ]
[0.72 0.81 0.68 0.71 0.72 0.71 0.76 0.71 0.75 0.77]
[0.716874   0.80942541 0.66444801 0.69744907 0.71259625 0.70409126
 0.74164665 0.70840203 0.74588517 0.76677245]
